<a href="https://colab.research.google.com/github/farahbakhsh3/X1/blob/master/nietzsche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, CuDNNLSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file('ferdosi.txt', 
                origin='https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt')
    
# path = get_file('nietzsche.txt', 
#                 origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])


corpus length: 2545108
total chars: 37
nb sequences: 848356
Vectorization...
Build model...
Epoch 1/60
848356/848356 [==============================] - 109s 128us/step - loss: 1.6743

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "زد سر از کوه تابنده شید
برآمد سر تاج روز"
زد سر از کوه تابنده شید
برآمد سر تاج روزی به جای
به آن باره بر سر به پیلید و راه
به نزدیک آن باره بر پاک باد
به آرام و با داد و با باره بود
بدو گفت کای شاد و با دانش و باد
به نزدیک او با به بازی به دست
به گرد و به فرمود و با داد و باد
به باری به بازی به بازار و باد
به بازی به پیمان به باران به گرد
برآمد ز تاران به نامی به راه
به دادی به باد و بدین داد و باد
به نامی به باران برآمد ز نام
که آن بارگاه بر آمد به درد
بدو گفت کای داد و بر داد
----- diversity: 0.5
----- Generating with seed: "زد سر از کوه تابنده شید
برآمد سر تاج روز"
زد سر از کوه تابنده شید
برآمد سر تاج روزی بگرد
بر آراست آن را به رای زمین
ز آمد برآید به شادی به داد
ازین نامداران دریا به هر
به آن چون به پیلان به بالا ز

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in log


ه روز اندر آمد به مرد
که شاهی بران نامداران بود
بیامد برانگیخت از ب
----- diversity: 1.0
----- Generating with seed: "هها بشمرد
که بااو ستارهشمر گفته بود
ز گف"
هها بشمرد
که بااو ستارهشمر گفته بود
ز گفتار آتش فراوان به نیست
که شد مهر فرمود هر هر کسی
زمین پیش او سیبمندند من
گر آییز با نامداران نوشت
کسی کرد راسرم و ایلان درش
نبین از به چشم ز اسبان به پیش
به میبان بیامد سلیری نهاد
برین عاغ شد رون به پیش آورد
پر از دیزبان خوارمم و تخور
توی را همی برزدیب پزر نبر
ب زر و زن زالی و در زمان
که هنداچه از گیو بایدش خرد
پری با مراپردم آمد مرا
بزرگانان چون پسندیده میو
ازان ناملر کورسی جام پیش
که داندش زرم ش
----- diversity: 1.2
----- Generating with seed: "هها بشمرد
که بااو ستارهشمر گفته بود
ز گف"
هها بشمرد
که بااو ستارهشمر گفته بود
ز گفتارتردان جهاندیشگان
مرا یادگاران نبودس نگاه
همان رون زیان من ونفرازآنم
مچون تیرها شاه و دیوان بوش
ز دایند یزدان یکی آختر
تو آمد ز بهشار و گردنک بود
چنین گفت کای دژ به سرز منست
سزرهن ستآره ستامش منیژ
بسان جهان از ز هر دو به ننز
بروشگ پشبگاه را کشوری
گر زان به کم ازیند